In [1]:
!pip install -q streamlit
!pip install langchain
!pip install pypdf
!pip install sentence_transformers
!pip install chromadb
!pip install accelerate
!pip install --upgrade accelerate
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
%%writefile app.py

import streamlit as st
import torch
from transformers import pipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain

def get_similar_docs(question, Vector_db, similar_doc_count):
    similar_docs = Vector_db.similarity_search(question, similar_doc_count)

    unique_docs = []
    seen_doc_contents = set()

    for doc in similar_docs:
        doc_content = doc.page_content  # Assuming 'page_content' is the key for the document content
        if doc_content not in seen_doc_contents:
            unique_docs.append(doc)
            seen_doc_contents.add(doc_content)

    return unique_docs

def build_qa_chain():
    torch.cuda.empty_cache()
    # Using databricks/dolly because mine OpenAI Credits are expired
    model_name = "databricks/dolly-v2-3b" # can use dolly-v2-3b or dolly-v2-7b for smaller model and faster inferences.
    # Increase max_new_tokens for a longer response, since I am on my pc I will be using only 256
    # Other settings might give better results! Play around
    instruct_pipeline = pipeline(model=model_name, device_map="auto", trust_remote_code=True)
    prompt = PromptTemplate(input_variables=['context', 'question'], template=prompt_template)

    hf_pipe = HuggingFacePipeline(pipeline=instruct_pipeline)
    return load_qa_chain(llm=hf_pipe, chain_type="stuff", prompt=prompt, verbose=True)

def answer_question(question, Vector_db):
    similar_docs = get_similar_docs(question, Vector_db, similar_doc_count=4)
    result = qa_chain({"input_documents": similar_docs, "question": question})

    if result is None:
        return

    return result['output_text']

prompt_template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

  Instruction:
  Use only information in the following paragraphs to answer the question at the end.
  Explain the answer with reference to these paragraphs.
  If you don't have the information in below paragraphs then give response "I will get back to you on this".

  {context}

  Question: {question}

  Response:
  """

# Setup the app title
st.title('Ask AI')

# Interface for uploading documents
pdf_files = st.file_uploader("Upload PDF files", accept_multiple_files=True)
Vector_db = None
if pdf_files:
    # Process uploaded documents
    all_documents = []
    for pdf_file in pdf_files:
        loader = PyPDFLoader(pdf_file.name)
        documents = loader.load()
        all_documents.extend(documents)

    # Split documents and create vector store
    text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=40)
    all_documents = text_splitter.split_documents(all_documents)
    hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")
    Vector_db = Chroma.from_documents(collection_name="document_docs", documents=all_documents, embedding=hf_embed, persist_directory="Persist_dir")
    Vector_db.persist()

# Initialize question-answering pipeline
qa_chain = build_qa_chain()

# Setup a session state message variable to hold all the old messages
if 'messages' not in st.session_state:
    st.session_state.messages = []

# Display all the historical messages |
for message in st.session_state.messages:
    st.chat_message(message['role']).markdown(message['content'])

# Build a prompt input template to display the prompts
prompt = st.chat_input('Pass Your Prompt here')
# If the user hits enter then

if prompt:
    # Display the prompt
    st.chat_message('user').markdown(prompt)
    st.session_state.messages.append({'role': 'user', 'content':prompt})
    answer = answer_question(prompt, Vector_db)
    st.chat_message('assistant').markdown(answer)
    st.session_state.messages.append({
        'role': 'assistant',
        'content': answer
    })


Writing app.py


In [3]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.276s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.222.157.224
npx: installed 22 in 1.842s
your url is: https://easy-oranges-lay.loca.lt
